# ML module 1

In [16]:
'''
1) Загрузка данных
2) Исследование данных
3) Подготовка данных к обучению модели
4) Моделирование

'''

'\n1) Загрузка данных\n2) Исследование данных\n3) Подготовка данных к обучению модели\n4) Моделирование\n\n'

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing

## 1) Загрузка данных

In [29]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

display(train.head())
display(test.head())

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


## 2) EDA

### Типы данных 

In [22]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

### Количество уникальных значений в разных полях

In [24]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

### Проверка пропусков

In [34]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [40]:
'''
Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - численные значения
PassengerId, Cabin, Name - строки
HomePlanet, CryoSleep, Destination, VIP - категориальные значения

Transported - целевое значение
'''

'\nAge, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - целочисленные значения\nPassengerId, Cabin, Name - строки\nHomePlanet, CryoSleep, Destination, VIP - категориальные значения\n\nTransported - целевое значение\n'

## 3) Подготовка данных

### Заполнение пропусков

In [190]:
# Список колонок, которые заполню null
exclude_cols = ['PassengerId', 'Cabin', 'Name', 'Age']

# Заполнение пропусков
for col in train.columns:
    if col in exclude_cols: # Пропускаем эти колонки
        continue

    if train[col].dtype == 'object':  # Категориальные переменные
        train[col] = train[col].fillna(train[col].mode()[0]) # Cамое часто встречающееся
    else:  # Числовые переменные
        train[col] = train[col].fillna(train[col].median()) # Среднее значение после сортировки

mean_age = train['Age'].mean() # Среднее арифметическое
train['Age'] = train['Age'].fillna(mean_age)

train['Cabin'] = train['Cabin'].fillna('None')
train['Name'] = train['Name'].fillna('None')

train.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
Cost            0
haveCost        0
Deck            0
Num             0
Side            0
dtype: int64

### Добавляем колонки

In [75]:
# Добавляю колонку трат 
cost_col = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
train['Cost'] = train[cost_col].sum(axis=1)

train['haveCost'] = train['Cost'] > 0 

train[['Cost', 'haveCost']].head()

,Cost,haveCost
0,0.0,False
1,736.0,True
2,10383.0,True
3,5176.0,True
4,1091.0,True


In [91]:
# Разделяю колонку "Cabin" на три части: Deck, Num, Side
cabin = train['Cabin'].str.split('/', expand=True)

# Присваиваем правильные имена колонок
cabin.columns = ['Deck', 'Num', 'Side']

# Добавляем их в основной DataFrame
train['Deck'] = cabin['Deck']
train['Num'] = cabin['Num']
train['Side'] = cabin['Side']

# Заполняю пропуски
train['Num'] = train['Num'].fillna('None')
train['Side'] = train['Side'].fillna('None')

# Проверяем результат
train[['Cabin', 'Deck', 'Num', 'Side']].head()

,Cabin,Deck,Num,Side
0,B/0/P,B,0,P
1,F/0/S,F,0,S
2,A/0/S,A,0,S
3,A/0/S,A,0,S
4,F/1/S,F,1,S


### Обрабатываем по разному разные типы данных

In [110]:
# Делим столбцы, с которыми будем работать по категориям

cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'haveCost', 'Deck', 'Side']
num_cols = ['Age', 'Cost', 'Num']

#### Категориальные значения

In [98]:
from sklearn.preprocessing import OneHotEncoder

In [112]:
train[cat_cols]

,HomePlanet,CryoSleep,Destination,VIP,haveCost,Deck,Side
0,Europa,False,TRAPPIST-1e,False,False,B,P
1,Earth,False,TRAPPIST-1e,False,True,F,S
2,Europa,False,TRAPPIST-1e,True,True,A,S
3,Europa,False,TRAPPIST-1e,False,True,A,S
4,Earth,False,TRAPPIST-1e,False,True,F,S
...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,True,True,A,P
8689,Earth,True,PSO J318.5-22,False,False,G,S
8690,Earth,False,TRAPPIST-1e,False,True,G,S
8691,Europa,False,55 Cancri e,False,True,E,S


In [114]:
train[cat_cols[0]].value_counts()

HomePlanet
Earth     4803
Europa    2131
Mars      1759
Name: count, dtype: int64

In [116]:
ohe = OneHotEncoder(sparse_output=False)

ohe.fit(train[cat_cols])

OneHotEncoder(sparse_output=False)

In [118]:
display(ohe.categories_)

cat_col_array = np.concatenate(ohe.categories_)
cat_col_array

[array(['Earth', 'Europa', 'Mars'], dtype=object),
 array([False,  True]),
 array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e'], dtype=object),
 array([False,  True]),
 array([False,  True]),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'None', 'T'], dtype=object),
 array(['None', 'P', 'S'], dtype=object)]

array(['Earth', 'Europa', 'Mars', False, True, '55 Cancri e',
       'PSO J318.5-22', 'TRAPPIST-1e', False, True, False, True, 'A', 'B',
       'C', 'D', 'E', 'F', 'G', 'None', 'T', 'None', 'P', 'S'],
      dtype=object)

In [120]:
cat_cols_transform = ohe.transform(train[cat_cols])
cat_cols_transform

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [132]:
display(train[cat_cols].iloc[0])
display(cat_cols_transform[0])

HomePlanet          Europa
CryoSleep            False
Destination    TRAPPIST-1e
VIP                  False
haveCost             False
Deck                     B
Side                     P
Name: 0, dtype: object

array([0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0.])

#### Численнные значения

In [137]:
train[num_cols]

,Age,Cost,Num
0,39.0,0.0,0
1,24.0,736.0,0
2,58.0,10383.0,0
3,33.0,5176.0,0
4,16.0,1091.0,1
...,...,...,...
8688,41.0,8536.0,98
8689,18.0,0.0,1499
8690,26.0,1873.0,1500
8691,32.0,4637.0,608


In [139]:
train[num_cols].isna().sum()

Age     0
Cost    0
Num     0
dtype: int64

## 4) Моделирование

In [152]:
from catboost import CatBoostClassifier

In [154]:
numeric = train[num_cols].values
display('Численные переменные',
        numeric)

display('Закодированные категориальные переменные',
        cat_cols_transform)

X = np.concatenate([numeric,
                    cat_cols_transform], axis=1)

X_columns = num_cols + list(cat_col_array)
display(X_columns)

y = train['Transported'].values

'Численные переменные'

array([[39.0, 0.0, '0'],
       [24.0, 736.0, '0'],
       [58.0, 10383.0, '0'],
       ...,
       [26.0, 1873.0, '1500'],
       [32.0, 4637.0, '608'],
       [44.0, 4826.0, '608']], dtype=object)

'Закодированные категориальные переменные'

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

['Age',
 'Cost',
 'Num',
 'Earth',
 'Europa',
 'Mars',
 False,
 True,
 '55 Cancri e',
 'PSO J318.5-22',
 'TRAPPIST-1e',
 False,
 True,
 False,
 True,
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'None',
 'T',
 'None',
 'P',
 'S']

### Делим выборки

In [160]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)

display(X_train.shape,
        X_valid.shape,
        y_train.shape,
        y_valid.shape)

(6954, 27)

(1739, 27)

(6954,)

(1739,)

### Accuracy

In [233]:
model = CatBoostClassifier(
    random_seed=63,
    iterations=1000,
    learning_rate=0.03,
    l2_leaf_reg=3,
    bagging_temperature=1,
    random_strength=1,
    one_hot_max_size=2,
    leaf_estimation_method='Newton' 
)

model.fit(
    X_train, y_train,
    verbose=False,
    eval_set=(X_valid, y_valid)
)

In [176]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

y_pred = model.predict(X_valid)

In [179]:
sum(y_pred == y_valid)/ y_pred.shape

array([0.76078206])

In [181]:
print(accuracy_score(y_valid, y_pred))
confusion_matrix(y_valid, y_pred)

0.7607820586543991


array([[710, 151],
       [265, 613]])

In [183]:
print (classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

       False       0.73      0.82      0.77       861
        True       0.80      0.70      0.75       878

    accuracy                           0.76      1739
   macro avg       0.77      0.76      0.76      1739
weighted avg       0.77      0.76      0.76      1739



## 5) Предсказание тестовой выборки

In [186]:
display()
test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [196]:
# Заполняем пропуски тестовых данных
exclude_cols_test = ['PassengerId', 'Cabin', 'Name', 'Age']

for col_test in test.columns:
    if col_test in exclude_cols_test:
        continue

    if test[col_test].dtype == 'object':
        test[col_test] = test[col_test].fillna(test[col_test].mode()[0]).infer_objects(copy=False)
    else: 
        test[col_test] = test[col_test].fillna(test[col_test].median()) 

mean_age_test = test['Age'].mean()
test['Age'] = test['Age'].fillna(mean_age_test)

test['Cabin'] = test['Cabin'].fillna('None')
test['Name'] = test['Name'].fillna('None')

cost_col_test = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
test['Cost'] = test[cost_col_test].sum(axis=1)

test['haveCost'] = test['Cost'] > 0 

cabin_test = test['Cabin'].str.split('/', expand=True)

cabin_test.columns = ['Deck', 'Num', 'Side']

test['Deck'] = cabin_test['Deck']
test['Num'] = cabin_test['Num']
test['Side'] = cabin_test['Side']

test['Num'] = test['Num'].fillna('None')
test['Side'] = test['Side'].fillna('None')

test.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Cost            0
haveCost        0
Deck            0
Num             0
Side            0
dtype: int64

In [198]:
cat_cols_transform_test = ohe.transform(test[cat_cols])

numeric_test = test[num_cols].values

X_test = np.concatenate([numeric_test,
                        cat_cols_transform_test], axis=1)

predictions = model.predict(X_test)
predictions

array([ True, False,  True, ...,  True, False,  True])

In [202]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)

result = pd.read_csv('submission.csv')
result.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
